In [71]:
from models import scribbler 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
models=scribbler.Scribbler(4,3,32)

In [10]:
b=scribbler.ResidualBlock(32)

In [58]:
b=scribbler.UpsamplingBlock(4,4,3,2,1)

In [28]:
for num,module in enumerate(models.children()):
    print module

Sequential (
  (conv_1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (norm_1): ReLU (inplace)
  (res_block_1): ResidualBlock (
    (resblock): Sequential (
      (conv_1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (norm_1): ReLU (inplace)
      (conv_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (batch_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (norm_2): ReLU (inplace)
  (res_block_2): ResidualBlock (
    (resblock): Sequential (
      (conv_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (norm_1): ReLU

In [73]:
import torch
from torch.autograd import Variable

test_var=torch.Tensor(1,4,128,128)
test_var=Variable(test_var)


seqblock=models._modules.items()
modules = seqblock[0][1]._modules.items()

In [74]:
inp = test_var
for num,name_module in enumerate(modules):
    name,module=name_module
    print name
    inp=module(inp)
    print inp.data.size()

conv_1
torch.Size([1, 32, 128, 128])
batch_1
torch.Size([1, 32, 128, 128])
norm_1
torch.Size([1, 32, 128, 128])
res_block_1
torch.Size([1, 32, 128, 128])
conv_2
torch.Size([1, 64, 64, 64])
batch_2
torch.Size([1, 64, 64, 64])
norm_2
torch.Size([1, 64, 64, 64])
res_block_2
torch.Size([1, 64, 64, 64])
conv_3
torch.Size([1, 128, 32, 32])
batch_3
torch.Size([1, 128, 32, 32])
norm_3
torch.Size([1, 128, 32, 32])
res_block_3
torch.Size([1, 128, 32, 32])
conv_4
torch.Size([1, 256, 16, 16])
batch_4
torch.Size([1, 256, 16, 16])
norm_4
torch.Size([1, 256, 16, 16])
res_block_4
torch.Size([1, 256, 16, 16])
res_block_5
torch.Size([1, 256, 16, 16])
res_block_6
torch.Size([1, 256, 16, 16])
res_block_7
torch.Size([1, 256, 16, 16])
res_block_8
torch.Size([1, 256, 16, 16])
upsampl_1
torch.Size([1, 128, 32, 32])
batch_5
torch.Size([1, 128, 32, 32])
norm_5
torch.Size([1, 128, 32, 32])
res_block_9
torch.Size([1, 128, 32, 32])
res_block_10
torch.Size([1, 128, 32, 32])
upsampl_2
torch.Size([1, 64, 64, 64])
bat

In [60]:
b(test_var).data.size()

torch.Size([1, 4, 2, 2])

In [66]:
import torch.nn as nn
est=nn.Conv2d(4,4,3,1,1)
ups =nn.UpsamplingBilinear2d(scale_factor=2)

In [67]:
ups(est(test_var)).data.size()


torch.Size([1, 4, 256, 256])